In [ ]:
from datetime import datetime, timedelta
import urllib
import os
import io


def scrape_station(station):
    current_date = datetime(year=2006, month=1, day=1)
    end_date = datetime(year=2016, month=1, day=1)

    # Make sure a directory exists for the station web pages
    if not os.path.isdir('./%s' % (station)):
        os.mkdir(station)

    # Use .format(station, YYYY, M, D)
    lookup_URL = 'http://www.wunderground.com/history/airport/{}/{}/{}/{}/DailyHistory.html'

    while current_date != end_date:

        if current_date.day == 1:
            print(current_date)

        formatted_lookup_URL = lookup_URL.format(station,
                                                 current_date.year,
                                                 current_date.month,
                                                 current_date.day)
        html = urllib.urlopen(formatted_lookup_URL).read()

        out_file_name = '{}/{}-{}-{}.html'.format(station, current_date.year,
                                                  current_date.month,
                                                  current_date.day)

        with open(out_file_name,'w') as out_file:
            out_file.write(html)

        current_date += timedelta(days=1)

for station in ['KNYC']:
    scrape_station(station)

In [ ]:
from bs4 import BeautifulSoup
def parse_station(station, outfile):
    current_date = datetime(year=2006, month=1, day=1)
    end_date = datetime(year=2016, month=1, day=1)

    with open(outfile, 'a') as out_file:
    
        while current_date != end_date:
            with open('{}/{}-{}-{}.html'.format(station,
                                                current_date.year,
                                                current_date.month,
                                                current_date.day)) as in_file:
                soup = BeautifulSoup(in_file.read(), 'html.parser')

                try:
                    rows = soup.find(id='obsTable').find_all('tr', class_='no-metars')
                    for tr in rows:
                        cols = tr.findAll('td')
                        data = [c.text.replace('\n', '').replace('\t', '') for c in cols]
                        
                        if len(data) < 13:
                            data.insert(2, u'  -')        
                        
                        out_row = [station, '{}-{}-{}'.format(current_date.year, current_date.month, current_date.day)] + data
                        out_file.write('\t'.join(out_row).replace(u'\xa0', u' ').replace(u'\xb0', u'').encode('utf-8'))
                        out_file.write('\n')
                        
                except:
                    print "No history available for %s on %s" % (station, current_date)

                if current_date.day == 1:
                    print(current_date)
                    
                current_date += timedelta(days=1)

In [ ]:
with open('wunderground_weather.tsv', 'w') as out_file:
    out_file.write('station,date,time,temperature,heat_index,dew_point,humidity,pressure,visibility,'
                    'wind_dir,wind_speed,gust_speed,precip,events,conditions\n'.replace(',', '\t'))

In [ ]:
for station in ['KNYC']:
    print "Starting %s:" % (station)
    parse_station(station, 'wunderground_weather.tsv')
    
print "Done!"